<a href="https://colab.research.google.com/github/RasikaS24/Grover-s-Alogo-/blob/main/Copy_of_codefor115.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pennylane as qml
import matplotlib.pyplot as plt


def add_k_fourier(k, wires):
    for j in range(len(wires)):
        qml.RZ(k * np.pi / (2**j), wires=wires[j])


def multiplication(reg_x, reg_y, reg_z):
    # prepare sol-qubits to counting
    qml.QFT(wires=reg_z)

    # add m to the counter
    for i in range(len(reg_y)):
        for j in range(len(reg_x)):
            coeff = 2 ** (len(reg_x) + len(reg_y) - i - j - 2)
            qml.ctrl(add_k_fourier, control=[reg_y[i], reg_x[j]])(coeff, reg_z)

    # return to computational basis
    qml.adjoint(qml.QFT)(wires=reg_z)


# Number we want to factorize(a biprime number)
N=int(input("Enter a number: "))

# Number of qubits needed to encode N
n=np.ceil(np.log2(N))
n=int(n)

reg_z = [i for i in range(n)]   # qubits needed to encode the solution
reg_x = [i for i in range(n, 2*n-2)]                 # qubits needed to encode m
reg_y = [i for i in range(2*n-2, 3*n-3)]                 # qubits needed to encode k


dev = qml.device("default.qubit", wires=reg_x + reg_y + reg_z)

n_wires = len(dev.wires)

#create variable for number of iterations
number_of_iterations = int((np.pi/4)*np.sqrt(21))

@qml.qnode(dev)
def factorization(N, reg_x, reg_y, reg_z, number_of_iterations):
    # Superposition of the input
    for wire in reg_x:
        qml.Hadamard(wires=wire)

    for wire in reg_y:
        qml.Hadamard(wires=wire)

    for i in range(number_of_iterations):
    # Apply the multiplication
      multiplication(reg_x, reg_y, reg_z)

    # Change sign of n
      qml.FlipSign(N, wires=reg_z)

    # Uncompute multiplication
      qml.adjoint(multiplication)(reg_x, reg_y, reg_z)

    # Apply Grover operator
      qml.GroverOperator(wires=reg_x + reg_y)

    return qml.probs(wires=reg_x)

qml.draw_mpl(factorization, show_all_wires=True)(N, reg_x, reg_y, reg_z, 1)
plt.show()

p = np.argmax(factorization(N, reg_x, reg_y, reg_z, number_of_iterations))
q = np.argmax(factorization(N, reg_y, reg_x, reg_z, number_of_iterations))

if p>=q:
  p = N//q
else:
  q = N//p

print(p)
print(q)

Enter a number: 115


5
23
